# Prepare

In [1]:
!nvidia-smi

Mon Dec 19 04:03:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%cd "/content/drive/MyDrive/CE Project"
!ls

/content/drive/MyDrive/CE Project
 data   softmax.ipynb   svm2.ipynb   SVM.ipynb	'Use Library.ipynb'


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import time
from sklearn.metrics import accuracy_score, f1_score
from keras.datasets import mnist, cifar10
from sklearn.model_selection import train_test_split
import torch
from sklearn.preprocessing import OneHotEncoder
import random

In [4]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed_everything(22)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define model

In [6]:
def svm_loss(W, X, y, reg):
    d, C = W.shape
    N = X.shape[0]
    loss = 0
    dW = torch.zeros_like(W)
    Z = torch.mm(X, W)
    id0 = range(Z.shape[0])
    X = X.cpu().numpy()
    W = W.cpu().numpy()
    Z = Z.cpu().numpy()
    y = y.cpu().numpy().astype(int)
    correct_class_score = Z[id0, y].reshape(N, 1)
    margins = np.maximum(0, Z - correct_class_score + 1)
    margins[id0, y] = 0
    loss = np.sum(margins)
    loss /= N
    loss += 0.5 * reg * np.sum(W*W)

    F = (margins > 0).astype(int)
    F[np.arange(F.shape[0]), y] = np.sum(-F, axis=1)
    dW = X.T.dot(F)/N + reg*W
    return loss, torch.from_numpy(dW).to(device)

def svm_fit(X, y, Winit, reg, lr=.1, batch_size=32, epochs=100, print_every=5):
    W = torch.clone(Winit)
    loss_hist = []
    n_batches = int(np.ceil(X.shape[0]/batch_size))

    for ep in range(epochs):
        mix_ids = torch.randperm(X.shape[0])
        
        for i in range(n_batches):
            ids = mix_ids[batch_size*i : min(batch_size*(i+1), X.shape[0])]
            X_batch = X[ids]
            y_batch = y[ids]
            loss_i, dW = svm_loss(W, X_batch, y_batch, reg)
            loss_hist.append(loss_i)
            W -= lr*dW

        if ep % print_every == 0 and ep > 0:
            print('Epoch:', ep, 'loss:', loss_hist[-1])
    return W, loss_hist

def svm_predict(W, X):
    Z = X.dot(W)
    return np.argmax(Z, axis=1)

def svm_evaluate(W, X, y):
    y_pred = svm_predict(W, X)
    acc = 100*np.mean(y_pred==y)
    return acc

# MNIST

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(y_train[0])
# le = OneHotEncoder()
# y_train = le.fit_transform(y_train.reshape(-1, 1)).toarray()
# y_test = le.transform(y_test.reshape(-1, 1)).toarray()
# print(y_train[0], y_test[0])

X_train = X_train.reshape(-1, 784)
X_train = X_train.astype('float32')
X_test = X_test.reshape(-1, 784)
X_test = X_test.astype('float32')

# normalize
X_train /= 255
X_test /= 255
print('Train samples:', X_train.shape)
print('Test samples:', X_test.shape)

5
Train samples: (60000, 784)
Test samples: (10000, 784)


In [ ]:
X_train = torch.from_numpy(X_train).to(device)
y_train = torch.from_numpy(y_train).to(device)
# X_test = torch.from_numpy(X_test).to(device)
# y_test = torch.from_numpy(y_test).to(device)

In [ ]:
W_init = torch.randn(X_train.shape[1], 10).to(device)

### lambda = 0.001

In [ ]:
lr = 0.01
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=0.001, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 5.23287737083435
Epoch: 10 loss: 5.445091423034668
Epoch: 15 loss: 2.546519454956055
Epoch: 20 loss: 2.553481315612793
Epoch: 25 loss: 2.317629577636719
Epoch: 30 loss: 1.508632577419281
Epoch: 35 loss: 0.9057253487706185
Epoch: 40 loss: 1.4887964224815369
Epoch: 45 loss: 0.8017264397144318
Epoch: 50 loss: 0.7062967519760133
Epoch: 55 loss: 0.6829353022575378
Epoch: 60 loss: 0.40037004935741427
Epoch: 65 loss: 1.1924680876731872
Epoch: 70 loss: 0.7328097715377808
Epoch: 75 loss: 0.5090261018276214
Epoch: 80 loss: 0.3310925956964493
Epoch: 85 loss: 0.21371076822280885
Epoch: 90 loss: 0.2804347641468048
Epoch: 95 loss: 0.6075698838233947
Train completed in 122.44973039627075s


In [ ]:
svm_evaluate(W.cpu().numpy(), X_test, y_test)

92.15

### lambda = 2

In [ ]:
lr = 0.01
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=2, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 2.997628331184387
Epoch: 10 loss: 3.335686445236206
Epoch: 15 loss: 2.767026901245117
Epoch: 20 loss: 2.8246480226516724
Epoch: 25 loss: 2.8158308267593384
Epoch: 30 loss: 3.0921460390090942
Epoch: 35 loss: 3.11791729927063
Epoch: 40 loss: 2.464823007583618
Epoch: 45 loss: 2.4028929471969604
Epoch: 50 loss: 2.893133044242859
Epoch: 55 loss: 2.3990789651870728
Epoch: 60 loss: 2.784099817276001
Epoch: 65 loss: 3.2218668460845947
Epoch: 70 loss: 2.5133384466171265
Epoch: 75 loss: 2.9860739707946777
Epoch: 80 loss: 3.195020914077759
Epoch: 85 loss: 2.974164605140686
Epoch: 90 loss: 2.565989375114441
Epoch: 95 loss: 2.59869122505188
Train completed in 122.24206471443176s


In [ ]:
svm_evaluate(W.cpu().numpy(), X_test, y_test)

84.5

### lambda = 0.0001

In [ ]:
lr = 0.01
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=0.0001, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 4.866084451293945
Epoch: 10 loss: 0.586824250149727
Epoch: 15 loss: 1.6677828102111816
Epoch: 20 loss: 0.8152812584877014
Epoch: 25 loss: 1.3839889667510987
Epoch: 30 loss: 1.047876100730896
Epoch: 35 loss: 0.8941404298782349
Epoch: 40 loss: 0.34818168756961826
Epoch: 45 loss: 0.77340208568573
Epoch: 50 loss: 0.8789427742958069
Epoch: 55 loss: 0.783569769191742
Epoch: 60 loss: 0.6571780781745911
Epoch: 65 loss: 0.5833093968391418
Epoch: 70 loss: 0.2620575168371201
Epoch: 75 loss: 2.1251307676315307
Epoch: 80 loss: 0.8791972526550293
Epoch: 85 loss: 0.517705661725998
Epoch: 90 loss: 1.166945099925995
Epoch: 95 loss: 0.7504736161231995
Train completed in 119.65892601013184s


In [ ]:
svm_evaluate(W.cpu().numpy(), X_test, y_test)

90.42999999999999

# Flower

In [ ]:
# Đường dẫn
features_path = '/content/drive/MyDrive/CE Project/data/flower/flower_features.sav'
labels_path = '/content/drive/MyDrive/CE Project/data/flower/flower_labels.sav'

# Load dữ liệu
features = np.array(pickle.load(open(features_path, 'rb')))
labels = np.array(pickle.load(open(labels_path, 'rb')))

In [ ]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
max_values = np.max(X_train)
min_values = np.min(X_train)
a, b = 0, 1
X_train = a + (X_train - min_values) * (b-a) / (max_values - min_values)
X_test = a + (X_test - min_values) * (b-a) / (max_values - min_values)

In [ ]:
X_train = torch.from_numpy(X_train).to(device)
y_train = torch.from_numpy(y_train).to(device)

In [ ]:
W_init = torch.randn(X_train.shape[1], np.max(y_test)+1).to(device)

### lambda = 0.001

In [ ]:
lr = 1
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=0.001, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 142.34231898352505
Epoch: 10 loss: 101.29641978186369
Epoch: 15 loss: 72.10344203355909
Epoch: 20 loss: 51.31295171707869
Epoch: 25 loss: 36.5335450451076
Epoch: 30 loss: 26.020406749874354
Epoch: 35 loss: 18.527023953840136
Epoch: 40 loss: 13.19798716469109
Epoch: 45 loss: 9.409422608435154
Epoch: 50 loss: 6.716860382378101
Epoch: 55 loss: 4.8075468352139
Epoch: 60 loss: 3.4375584176033738
Epoch: 65 loss: 2.4715973189473153
Epoch: 70 loss: 1.7718094012886285
Epoch: 75 loss: 1.2907693586796523
Epoch: 80 loss: 0.9486841434687376
Epoch: 85 loss: 0.6857430469840765
Epoch: 90 loss: 0.5166416568011045
Epoch: 95 loss: 0.3801060227751732
Train completed in 35.47990155220032s


In [ ]:
svm_evaluate(W.cpu().numpy(), X_test, y_test)

93.01470588235294

### lambda = 1

In [ ]:
lr = 1
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=1, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 39.43503761291504
Epoch: 10 loss: 45.64558792114258
Epoch: 15 loss: 50.76962089538574
Epoch: 20 loss: 45.05560874938965
Epoch: 25 loss: 58.77431869506836
Epoch: 30 loss: 44.06486892700195
Epoch: 35 loss: 52.438804626464844
Epoch: 40 loss: 42.72726631164551
Epoch: 45 loss: 44.34421348571777
Epoch: 50 loss: 36.50621032714844
Epoch: 55 loss: 33.914899826049805
Epoch: 60 loss: 71.37447166442871
Epoch: 65 loss: 49.65016746520996
Epoch: 70 loss: 47.56419563293457
Epoch: 75 loss: 38.99336242675781
Epoch: 80 loss: 71.72906875610352
Epoch: 85 loss: 64.74276351928711
Epoch: 90 loss: 32.363969802856445
Epoch: 95 loss: 49.513750076293945
Train completed in 30.72786855697632s


In [ ]:
svm_evaluate(W.cpu().numpy(), X_test, y_test)

13.602941176470587

### lambda = 0.0001

In [ ]:
lr = 1
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=0.0001, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 20.631403677213193
Epoch: 10 loss: 19.968097904992103
Epoch: 15 loss: 19.16260625
Epoch: 20 loss: 18.5222703125
Epoch: 25 loss: 17.9032296875
Epoch: 30 loss: 17.30486875
Epoch: 35 loss: 16.7264453125
Epoch: 40 loss: 16.1673171875
Epoch: 45 loss: 15.62689375
Epoch: 50 loss: 15.10453125
Epoch: 55 loss: 14.599728125
Epoch: 60 loss: 14.111682812500002
Epoch: 65 loss: 13.6399734375
Epoch: 70 loss: 13.184046875
Epoch: 75 loss: 12.743403125
Epoch: 80 loss: 12.31750859375
Epoch: 85 loss: 11.9058046875
Epoch: 90 loss: 11.507850000000001
Epoch: 95 loss: 11.123241406250001
Train completed in 31.30860686302185s


In [ ]:
svm_evaluate(W.cpu().numpy(), X_test, y_test)

69.11764705882352

# VNnews

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import LabelEncoder

In [8]:
# Đường dẫn
train_x_path = '/content/drive/MyDrive/CE Project/data/vnnews/train_x.sav'
train_y_path = '/content/drive/MyDrive/CE Project/data/vnnews/train_y.sav'
test_x_path = '/content/drive/MyDrive/CE Project/data/vnnews/test_x.sav'
test_y_path = '/content/drive/MyDrive/CE Project/data/vnnews/test_y.sav'

# Load dữ liệu
X_train = np.array(pickle.load(open(train_x_path, 'rb')))
y_train = np.array(pickle.load(open(train_y_path, 'rb')))
X_test = np.array(pickle.load(open(test_x_path, 'rb')))
y_test = np.array(pickle.load(open(test_y_path, 'rb')))

In [9]:
feature_extractor = Pipeline([
    # Chuyển tập dữ liệu sang dạng vector count dựa trên một vocab chung (BoW), loại bỏ những word có tần suất xuất hiện > 0.8
    ('vectorize', CountVectorizer(max_df=0.8, encoding='utf-16')),
    
    # Áp dụng TF-IDF để trích chọn đặc trưng (extract feature)
    ('feature extracter', TfidfTransformer())
])

X_train = feature_extractor.fit_transform(X_train, y_train)
X_test = feature_extractor.transform(X_test)
print('Kích thước vector input: ', X_train.shape)
print('Kích thước vector test', X_test.shape)

Kích thước vector input:  (14375, 79170)
Kích thước vector test (12076, 79170)


In [10]:
X_train = X_train.astype('float32').toarray()
X_test = X_test.astype('float32').toarray()

In [11]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [12]:
X_train = torch.from_numpy(X_train).to(device)
y_train = torch.from_numpy(y_train).to(device)

In [13]:
W_init = torch.randn(X_train.shape[1], np.max(y_test)+1).to(device)

### lambda = 1

In [ ]:
lr = 1
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=1, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 35.94359261648995
Epoch: 10 loss: 35.97727475847517
Epoch: 15 loss: 35.69841412135533
Epoch: 20 loss: 34.17227077484131
Epoch: 25 loss: 33.966930934361045
Epoch: 30 loss: 37.664151191711426
Epoch: 35 loss: 36.34470026833671
Epoch: 40 loss: 35.215360505240305
Epoch: 45 loss: 35.962204115731375
Epoch: 50 loss: 34.730122021266396
Epoch: 55 loss: 37.83451802389962
Epoch: 60 loss: 34.120737757001606
Epoch: 65 loss: 37.5014659336635
Epoch: 70 loss: 34.26925264086042
Epoch: 75 loss: 36.843397412981304
Epoch: 80 loss: 36.24270643506731
Epoch: 85 loss: 32.79376002720424
Epoch: 90 loss: 35.93608883449009
Epoch: 95 loss: 36.23244966779436
Train completed in 1849.01012134552s


In [ ]:
svm_evaluate(W.cpu().numpy(), X_test, y_test)

30.548194766478964

### lambda=0.001

In [15]:
lr = 1
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=0.001, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 7.250808940342495
Epoch: 10 loss: 2.8676213618687223
Epoch: 15 loss: 1.6368440540858678
Epoch: 20 loss: 1.6988503576006209
Epoch: 25 loss: 1.0895723690305437
Epoch: 30 loss: 1.398463110787528
Epoch: 35 loss: 1.4499401498522078
Epoch: 40 loss: 1.6293098318917412
Epoch: 45 loss: 1.3921966318402972
Epoch: 50 loss: 2.744684691292899
Epoch: 55 loss: 1.627111733027867
Epoch: 60 loss: 1.6719393021719797
Epoch: 65 loss: 1.6304771614074707
Epoch: 70 loss: 1.3875493556431362
Epoch: 75 loss: 1.2569296464920043
Epoch: 80 loss: 1.7469698099408832
Epoch: 85 loss: 1.7879013617379325
Epoch: 90 loss: 1.6561872215270996
Epoch: 95 loss: 2.918875257219587
Train completed in 1970.2981507778168s


In [16]:
svm_evaluate(W.cpu().numpy(), X_test, y_test)

90.63431599867506

### lambda=0.0001

In [17]:
lr = 1
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=0.0001, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 63.17978241075789
Epoch: 10 loss: 40.38595361099243
Epoch: 15 loss: 25.421778125
Epoch: 20 loss: 16.268486838449753
Epoch: 25 loss: 10.606724580683027
Epoch: 30 loss: 6.744856212288993
Epoch: 35 loss: 4.369629995373318
Epoch: 40 loss: 2.833372330406734
Epoch: 45 loss: 2.1901251256125316
Epoch: 50 loss: 1.248574412904467
Epoch: 55 loss: 0.8560222677230835
Epoch: 60 loss: 0.6293635775293622
Epoch: 65 loss: 0.4687798723084586
Epoch: 70 loss: 0.34894357910156254
Epoch: 75 loss: 0.32942597524097994
Epoch: 80 loss: 0.2569248994282314
Epoch: 85 loss: 0.2709361138888768
Epoch: 90 loss: 0.21766380946295605
Epoch: 95 loss: 0.22539207819529944
Train completed in 1836.0166013240814s


In [18]:
svm_evaluate(W.cpu().numpy(), X_test, y_test)

90.65087777409738